# Exam December 2021

In [2]:
import numpy as np
from scipy.stats import norm

C:\Users\migue\AppData\Roaming\Python\Python38\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# normal distribution
N = norm.cdf

In [4]:
# Price at t=0
s0 = 109

# Volatility
sigma = 0.179

# rf
r = 0.035

# # of trading days
t_days = 250

# T and T as a % of t_days
T = 203
Tp = 203/t_days

# t and t as a % of t_days
t1 = 91
t1p = 91/t_days

# time step
dt = 1/t_days

# Q1.

Compute the price of a plain vanilla at-the-money European call option.

In [5]:
# Calculate price of call option

def bs_call(S_, X_, r_, Tp_, vol_):
    # Calculate d1
    d1 = (np.log(S_/X_)+(r_+(vol_**2)/2)*Tp_)/(vol_*np.sqrt(Tp_))
    # Calculate d2
    d2 = d1 - vol_ * np.sqrt(Tp_)
    # Calculate call value
    call = S_ * N(d1) - X_ * np.exp(-r_ * Tp_) * N(d2)

    print("S: ", S_)
    print("K: ", X_)
    print("r: ", r_)
    print("T: ", Tp_)
    print("sigma: ", vol_)
    print("d1: ", d1)
    print("d2: ", d2)
    print("N(d1): ", N(d1))
    print("N(d2): ", N(d2), "\n")
    print("Call Value: ")

    return call

In [6]:
bs_call(s0, s0, r, Tp, sigma)

S:  109
K:  109
r:  0.035
T:  0.812
sigma:  0.179
d1:  0.25684415918489106
d2:  0.0955453929155857
N(d1):  0.6013504604192399
N(d2):  0.5380591815915454 

Call Value: 


8.54207608388203

# Q2.

Compute the price of a plain vanilla at-the-money European put option.

In [7]:
# Calculate price of put option

def bs_put(S_, X_, r_, Tp_, vol_):
    # Calculate d1
    d1 = (np.log(S_/X_)+(r_+(vol_**2)/2)*Tp_)/(vol_*np.sqrt(Tp_))
    # Calculate d2
    d2 = d1 - vol_ * np.sqrt(Tp_)
    # Calculate call value
    put = X_ * np.exp(-r_ * Tp_) * N(-d2) - S_ * N(-d1)

    print("S: ", S_)
    print("K: ", X_)
    print("r: ", r_)
    print("T: ", Tp_)
    print("sigma: ", vol_)
    print("d1: ", d1)
    print("d2: ", d2)
    print("N(-d1): ", N(-d1))
    print("N(-d2): ", N(-d2), "\n")
    print("Put Value: ")

    return put

In [8]:
bs_put(s0, s0, r, Tp, sigma)

S:  109
K:  109
r:  0.035
T:  0.812
sigma:  0.179
d1:  0.25684415918489106
d2:  0.0955453929155857
N(-d1):  0.39864953958076016
N(-d2):  0.46194081840845463 

Put Value: 


5.487901472824426

# Q3. & Q4.

3. Compute the price of a call option that matures at time 𝑇, and whose strike price (𝐾) is only known at time 𝑡, such that:
• 𝑲 = 𝐦𝐚𝐱{𝑺𝟏, 𝑺𝟐, … , 𝑺𝒕}

4. Compute the price of a put option that matures at time 𝑇, and whose strike price (𝐾) is only known at time 𝑡, such that:
• 𝑲 = 𝐦𝐢𝐧{𝑺𝟏, 𝑺𝟐, … , 𝑺𝒕}

In [9]:
# Define function to construct price paths according to Ito's Lemma

def price_paths(S0_, r_, sigma_, T_, dt_, n_paths):
    paths = []
    for iteration in np.arange(0,n_paths,1):
        current_price = S0_
        iteration_path = [current_price]
        z = norm.rvs(size = T_)
        for step in np.arange(1, T_, 1):
            current_price = current_price * np.exp((r_ - ((sigma_**2)/2)) * dt_ + z[step-1] * sigma_ * np.sqrt(dt_))
            iteration_path.append(current_price)
        paths.append([iteration, iteration_path])
    return paths


In [10]:
# Construct price paths (adjustable number of iterations)

iter = 2000
paths = price_paths(s0, r, sigma, T, dt, iter)

In [11]:
# Just check if average price at time T makes sense

def average_price(paths_, time):
    sum = 0
    for path_ in range(0, len(paths_)):
        sum += paths_[path_][1][time-1]
    return sum/len(paths_)

average_price(paths, T)

112.87673261474117

In [12]:
# Define function to calculate call payoff

def call_payoff(sT_, K_):
    return max(sT_ - K_, 0)

In [13]:
# Define function to calculate put payoff

def put_payoff(sT_, K_):
    return max(K_ - sT_, 0)

In [14]:
discount_payoffs = np.exp(-r*Tp)

## Q3.

In [15]:
# For each path
payoff_list = []
for path in paths:
    sT = path[1][-1]
    K = np.max(path[1][0:t1-1])
    # Calculate payoff
    payoff = call_payoff(sT, K)
    # Add payoff to payoff list
    payoff_list.append(payoff)
print(np.average(payoff_list)*discount_payoffs)

3.723568137089798


## Q4.

In [16]:
# For each path
payoff_list = []
for path in paths:
    sT = path[1][-1]
    K = np.min(path[1][0:t1-1])
    # Calculate payoff
    payoff = put_payoff(sT, K)
    # Add payoff to payoff list
    payoff_list.append(payoff)
print(np.average(payoff_list)*discount_payoffs)

1.807267521683589


# Q5.

Compute the price of an option that allows you to choose between the options described in questions 3 and 4 at time 𝑡.


In [17]:
# For each path
payoff_list = []
for path in paths:
    sT = path[1][-1]
    st1 = path[1][t1-1]
    #print("st1: ", st1)
    K3 = np.max(path[1][0:t1-1])
    #print("K3: ", K3)
    K4 = np.min(path[1][0:t1-1])
    #print("K4: ", K4)
    # Choose the one that is more in the money at time t
    # If call payoff is larger than put at t, choose call
    if st1 - K3 >= K4 - st1:
        #print("<Call>")
        payoff = call_payoff(sT, K3)
    # If put payoff is larger than call at t, choose put
    elif st1 - K3 < K4 - st1:
        #print("<Put>")
        payoff = put_payoff(sT, K4)
    #print("sT: ", sT)
    #print("Payoff: ", payoff)
    #print("------------")
    payoff_list.append(payoff)
print(np.average(payoff_list)*discount_payoffs)

4.223458985385165
